In [36]:
import simpy
import random

import pandas as pd

class Atendimento:
    def __init__(self, env, num_atendentes, taxa_chegada, tempo_atendimento):
        self.env = env
        self.num_atendentes = num_atendentes
        self.taxa_chegada = taxa_chegada
        self.tempo_atendimento = tempo_atendimento
        self.fila = []
        self.atendentes = [None] * self.num_atendentes
        
        self.num_clientes_total = 0
        self.num_clientes_atendidos = 0
        self.tempo_espera_total = 0
        self.tempo_atendimento_total = 0
        
        self.proxima_chegada = self.gera_chegada()
        self.env.process(self.chegada_cliente())
        
    def gera_chegada(self):
        return random.expovariate(self.taxa_chegada)
    
    def gera_tempo_atendimento(self):
        return random.expovariate(1.0 / self.tempo_atendimento)
    
    def chegada_cliente(self):
        while True:
            # Aguarda a chegada do próximo cliente
            yield self.env.timeout(self.proxima_chegada)
            
            # Gera um novo cliente e adiciona à fila
            self.num_clientes_total += 1
            nome = f"Cliente {self.num_clientes_total}"
            cliente = {'nome': nome, 'hora_chegada': self.env.now, 'hora_atendimento': None, 'tempo_espera': 0}
            self.fila.append(cliente)
            print(f"{nome} chegou ao sistema no tempo {self.env.now:.2f} minutos e entrou na fila.")
            print("Fila de espera:", [cliente['nome'] for cliente in self.fila]) # adiciona essa linha para imprimir a fila
            
            # Inicia o atendimento se houver atendentes disponíveis
            if None in self.atendentes:
                self.env.process(self.execucao_atendimento())

            # Agenda a próxima chegada
            self.proxima_chegada = self.gera_chegada()
            
    def execucao_atendimento(self):
        # Seleciona um atendente disponível
        atendente = self.atendentes.index(None)
        
        # Seleciona o próximo cliente da fila
        cliente = self.fila.pop(0)
        nome = cliente['nome']
        
        # Inicia o atendimento
        self.atendentes[atendente] = nome
        cliente['hora_atendimento'] = self.env.now
        tempo_atendimento = self.gera_tempo_atendimento()
        yield self.env.timeout(tempo_atendimento)
        self.atendentes[atendente] = None

        # Atualiza as estatísticas do sistema
        cliente['tempo_espera'] = cliente['hora_atendimento'] - cliente['hora_chegada']
        self.num_clientes_atendidos += 1
        self.tempo_espera_total += cliente['tempo_espera']
        self.tempo_atendimento_total += tempo_atendimento
        print(f"{nome} foi atendido pelo atendente {atendente + 1} no tempo {self.env.now:.2f} minutos.")


    # Verifica se o cliente desistiu da fila
    if random.uniform(0, 1) < prob_desistencia and any(cliente['nome'] == nome for cliente in fila):
        fila = [cliente for cliente in fila if cliente['nome'] != nome]
        print(f"{nome} desistiu da fila no tempo {env.now:.2f} minutos.")

In [37]:
env = simpy.Environment()
atendimento = Atendimento(env, num_atendentes=2, taxa_chegada=10/60, tempo_atendimento=5)
env.run(until=8*60)

Cliente 1 chegou ao sistema no tempo 9.17 minutos e entrou na fila.
Fila de espera: ['Cliente 1']
Cliente 2 chegou ao sistema no tempo 17.34 minutos e entrou na fila.
Fila de espera: ['Cliente 2']
Cliente 2 foi atendido pelo atendente 2 no tempo 17.78 minutos.
Cliente 1 foi atendido pelo atendente 1 no tempo 19.01 minutos.
Cliente 3 chegou ao sistema no tempo 22.47 minutos e entrou na fila.
Fila de espera: ['Cliente 3']
Cliente 3 foi atendido pelo atendente 1 no tempo 24.01 minutos.
Cliente 4 chegou ao sistema no tempo 27.59 minutos e entrou na fila.
Fila de espera: ['Cliente 4']
Cliente 4 foi atendido pelo atendente 1 no tempo 29.64 minutos.
Cliente 5 chegou ao sistema no tempo 31.29 minutos e entrou na fila.
Fila de espera: ['Cliente 5']
Cliente 6 chegou ao sistema no tempo 34.00 minutos e entrou na fila.
Fila de espera: ['Cliente 6']
Cliente 7 chegou ao sistema no tempo 34.21 minutos e entrou na fila.
Fila de espera: ['Cliente 7']
Cliente 6 foi atendido pelo atendente 2 no tempo 35.